# *Date Fruit - CNN Model*

*Bu projede, hurma meyvesi görüntülerini analiz ederek türüne göre sınıflandıran bir CNN Modeli geliştireceğiz. Modeli kaydedip Streamlit uygulamasına çevireceğiz ve HugginFace de çalışır hale getireceğiz.*

### *İmport Libraries*

In [1]:
import os                  # Dosya ve dizin işlemleri için
import random              # Rastgele seçimler yapmak için
import warnings            # Uyarı mesajlarını yönetmek için
warnings.filterwarnings('ignore') # Tüm uyarı mesajlarını görmezden gelmek için filterwarnings ile 'ignore' ayarını yapıyoruz

import numpy as np         # Sayısal hesaplamalar ve matris işlemleri
import pandas as pd        # Veri analizi ve tablo işlemleri
import cv2                 # Görüntü işleme ve video/frame işlemleri
import kagglehub           # Kaggle veri setlerini kolayca çekmek için
import random

from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split  # Veriyi eğitim/test setlerine ayırmak için

from tensorflow.keras.models import Model            # Keras Functional API model tanımı
from tensorflow.keras.layers import Flatten, Dense   # Katmanlar: Flatten ve tam bağlantılı Dense
from tensorflow.keras.applications import VGG16      # Önceden eğitilmiş VGG16 modeli
from tensorflow.keras.preprocessing.image import ImageDataGenerator

### *İmport Data*

In [2]:
# Sınıf etiketlerini bir liste halinde tanımlıyoruz
# 'Black Rot' -> Black Rot örnekleri
# 'ESCA' -> ESCA örnekleri
# 'Healthy' -> Healthy örnekleri
# 'Leaf Blight' -> Leaf Blight örnekleri
labels = ['Ajwa', 'Galaxy', 'Medjool', 'Meneifi', 'Nabtat Ali', 'Rutab', 'Shaishe', 'Sokari', 'Sugaey']

# Görüntülerin bulunduğu ana dizin yolunu belirtiyoruz
# Download latest version
path = kagglehub.dataset_download("wadhasnalhamdan/date-fruit-image-dataset-in-controlled-environment")

print("Path to dataset files:", path)

100%|██████████| 3.11G/3.11G [00:23<00:00, 143MB/s]

Extracting files...


Path to dataset files: /root/.cache/kagglehub/datasets/wadhasnalhamdan/date-fruit-image-dataset-in-controlled-environment/versions/1


In [19]:
img_path = os.path.join(path)
os.listdir(img_path)

['Galaxy',
 'Nabtat Ali',
 'Sugaey',
 'Rutab',
 'Medjool',
 'Ajwa',
 'Sokari',
 'Shaishe',
 'Meneifi']

### *CNN with Transfer Learning*

In [20]:
# Modelin girdi boyutu
img_width, img_height = 224, 224

# --- Veri hazırlama ---
train_datagen = ImageDataGenerator(rescale=1/255, validation_split=0.20)

train_datagenerator = train_datagen.flow_from_directory(
    directory=img_path,
    target_size=(img_width, img_height),
    class_mode='categorical',
    subset='training'
)

test_datagenerator = train_datagen.flow_from_directory(
    directory=img_path,
    target_size=(img_width, img_height),
    class_mode='categorical',
    subset='validation'
)

# --- Transfer Learning: VGG16 base ---
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(img_width, img_height, 3))
base_model.trainable = False  # Tüm katmanları dondur

# --- Functional API ile üst katmanlar ---
x = base_model.output
x = Flatten()(x)
x = Dense(1024, activation='relu')(x)
outputs = Dense(9, activation='softmax')(x)  # 5 sınıf

# --- Model tanımı ---
model = Model(inputs=base_model.input, outputs=outputs)

# --- Compile ---
model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

# --- Eğitim ---
model.fit(
    train_datagenerator,
    epochs=10,
    validation_data=test_datagenerator
)

Found 1329 images belonging to 9 classes.
Found 329 images belonging to 9 classes.
58889256/58889256 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
Epoch 1/10
42/42 ━━━━━━━━━━━━━━━━━━━━ 155s 3s/step - accuracy: 0.3020 - loss: 6.9516 - val_accuracy: 0.7599 - val_loss: 0.6229
Epoch 2/10
42/42 ━━━━━━━━━━━━━━━━━━━━ 115s 3s/step - accuracy: 0.8317 - loss: 0.4655 - val_accuracy: 0.8693 - val_loss: 0.3135
Epoch 3/10
42/42 ━━━━━━━━━━━━━━━━━━━━ 114s 3s/step - accuracy: 0.9333 - loss: 0.2357 - val_accuracy: 0.9149 - val_loss: 0.2991
Epoch 4/10
42/42 ━━━━━━━━━━━━━━━━━━━━ 113s 3s/step - accuracy: 0.9507 - loss: 0.1781 - val_accuracy: 0.8997 - val_loss: 0.3022
Epoch 5/10
42/42 ━━━━━━━━━━━━━━━━━━━━ 114s 3s/step - accuracy: 0.9484 - loss: 0.1616 - val_accuracy: 0.9483 - val_loss: 0.1931
Epoch 6/10
42/42 ━━━━━━━━━━━━━━━━━━━━ 109s 3s/step - accuracy: 0.9790 - loss: 0.0873 - val_accuracy: 0.9422 - val_loss: 0.2005
Epoch 7/10
42/42 ━━━━━━━━━━━━━━━━━━━━ 110s 3s/step - accuracy: 0.9847 - loss: 0.0697 - val_accuracy: 0.9

### *Save the Learning Model*

In [21]:
model.save("date_fruit_model_TL.keras")  # Streamlit ve Hugging Face uyumlu

In [22]:
# Test/validation setinden bir batch (küme) görüntü ve etiketleri al
val_images, val_labels = next(test_datagenerator)

# Modeli kullanarak bu görüntüler için tahmin yap
preds = model.predict(val_images)

# One-hot encoding şeklindeki gerçek etiketleri sınıf indekslerine çevir
y_true = np.argmax(val_labels, axis=1)

# Tahmin edilen olasılıkları sınıf indekslerine çevir
y_pred = np.argmax(preds, axis=1)

# Sınıf bazlı doğruluk, precision, recall ve F1-score'u yazdır
# target_names parametresi ile her sınıfın adı gösterilir
print(classification_report(y_true, y_pred, target_names=labels))

1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 818ms/step
              precision    recall  f1-score   support

        Ajwa       1.00      1.00      1.00         4
      Galaxy       1.00      1.00      1.00         4
     Medjool       1.00      1.00      1.00         3
     Meneifi       1.00      1.00      1.00         3
  Nabtat Ali       0.67      1.00      0.80         2
       Rutab       1.00      0.67      0.80         3
     Shaishe       1.00      1.00      1.00         3
      Sokari       1.00      1.00      1.00         6
      Sugaey       1.00      1.00      1.00         4

    accuracy                           0.97        32
   macro avg       0.96      0.96      0.96        32
weighted avg       0.98      0.97      0.97        32



Bu projede hurma meyvesi görüntülerini analiz ederek türlerini sınıflandıran bir CNN Modeli geliştirdik.
Datayı kaggle üzerinden çektik.
Kendi datamızı önceden eğitilmiş VGG16 modeli ile birleştirip geliştirdik.
%99.46 başarı elde ettik.